# RAG

In [ ]:
from get_Sentence import querySentence
from chunk_Split import GetChunk
from sentence_Embedding import Embedding
import textwrap
from sentence_transformers import SentenceTransformer, util
from time import perf_counter as timer
import torch

file_path = 'data/DemoData.pdf'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# extract sentences into a list
text = querySentence(file_path)
list_of_sentence = text.extractSentence()
print(list_of_sentence)
# split doc into chunks
chunks = GetChunk(list_of_sentence)
chunk = chunks.splitChunks()

# embedding chunks
embedded = Embedding(chunk)
embeddedChunk, pages_and_chunks = embedded.EmbedModel()
print("Embedded chunks: \n", embeddedChunk)

def print_wrapped(text, wrap_length=80):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

def get_RAG():
    # query
    query = input('Input something to find: ')

    # 2. Embed the query to the same numerical space as the text examples
    embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", device=device) # choose the device to load the model to (note: GPU will often be *much* faster than CPU)
    query_embedding = embedding_model.encode(query, convert_to_tensor=True)

    # 3. Get similarity scores with the dot product (we'll time this for fun)
    start_time = timer()
    dot_scores = util.dot_score(a=query_embedding, b=embeddedChunk)[0]
    end_time = timer()
    
    rag_output = ""

    rag_output += f"Time take to get scores on {len(embeddedChunk)} embeddings: {end_time-start_time:.5f} seconds. \n"

    # 4. Get the top-k results (3)
    top_results_dot_product = torch.topk(dot_scores, k=3)

    # print("Results:")
    # Loop through zipped together scores and indicies from torch.topk
    assistant_messages = ""
    for score, idx in zip(top_results_dot_product[0], top_results_dot_product[1]):
        rag_output += f"Score: {score:.4f}\n"
        # Print relevant sentence chunk (since the scores are in descending order, the most relevant chunk will be first)
        rag_output += "Text: \n"
        rag_output += f"""{pages_and_chunks[idx]["sentence_chunk"]} \n"""
        # Print the page number too so we can reference the textbook further (and check the results)
        rag_output += f"Page number: {pages_and_chunks[idx]['page_number']} \n"
        assistant_messages += pages_and_chunks[idx]["sentence_chunk"]

    return query, assistant_messages, rag_output



/home/quyld3/RAG/RAGLLM/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[{'page_number': 0, 'page_char_count': 0, 'page_word_count': 1, 'page_sentence_count_raw': 1, 'page_token_count': 0.0, 'text': '', 'sentences': [], 'pages_sentence_count_spacy': 0}, {'page_number': 1, 'page_char_count': 0, 'page_word_count': 1, 'page_sentence_count_raw': 1, 'page_token_count': 0.0, 'text': '', 'sentences': [], 'pages_sentence_count_spacy': 0}, {'page_number': 2, 'page_char_count': 262, 'page_word_count': 77, 'page_sentence_count_raw': 2, 'page_token_count': 65.5, 'text': 'Database  Fundamentals        Neeraj Sharma, Liviu Perniu, Raul F. Chong, Abhishek Iyer, Chaitali Nandan,   Adi-Cristina Mitea, Mallarswami Nonvinkere, Mirela Danubianu                           A book for the community by the community  F I R S T  E D I T I O N', 'sentences': ['Database  Fundamentals        Neeraj Sharma, Liviu Perniu, Raul F. Chong, Abhishek Iyer, Chaitali Nandan,   Adi-Cristina Mitea, Mallarswami Nonvinkere, Mirela Danubianu                           A book for the community by the

# LLM (Gemini)

In [3]:
# LLM model
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("Gemini_API")

genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-1.5-flash")
query, assistant_messages, rag_output = get_RAG()

message = f"""
            "You are an AI assistant that answers questions.",
            "You are a very honest person, never make up answer from what you do not know. If you do not know the answer, say exactly I DO NOT KNOW!",
            "content": {query}

"""

response = model.generate_content(message).text

if bool("I DO NOT KNOW!" in response):
    message += assistant_messages
    rag_response = model.generate_content(message).text
    print_wrapped(rag_output)
    print("__________________________________LLM answer______________________________________")
    print_wrapped(rag_response)
else:
    print_wrapped(response)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat2 in method wrapper_CUDA_mm)

: 